<a href="https://www.kaggle.com/code/mohamedchahed/covid-19-xray-vgg19?scriptVersionId=139909235" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [2]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D,Flatten,BatchNormalization,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
import pandas as pd

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [8]:
covid_path = "/kaggle/input/covid-x-ray/data/1"
no_covid_path = "/kaggle/input/covid-x-ray/data/0"

covid = [covid_path + "/" + i for i in os.listdir(covid_path)]
no_covid = [no_covid_path + "/" + i for i in os.listdir(no_covid_path)]

file_paths = covid + no_covid

df = pd.DataFrame({'path': file_paths})
df["label"] = ["covid"]*len(covid) + ["no_covid"]*len(no_covid)
df

,path,label
0,/kaggle/input/covid-x-ray/data/1/sub-S11193_se...,covid
1,/kaggle/input/covid-x-ray/data/1/covid_4208.png,covid
2,/kaggle/input/covid-x-ray/data/1/COVID-1994.png,covid
3,/kaggle/input/covid-x-ray/data/1/COVID-1825.png,covid
4,/kaggle/input/covid-x-ray/data/1/sub-S10677_se...,covid
...,...,...
47717,/kaggle/input/covid-x-ray/data/0/non_COVID (66...,no_covid
47718,/kaggle/input/covid-x-ray/data/0/non_COVID (15...,no_covid
47719,/kaggle/input/covid-x-ray/data/0/Normal (8837)...,no_covid
47720,/kaggle/input/covid-x-ray/data/0/non_COVID (96...,no_covid


In [9]:
from sklearn.model_selection import train_test_split
train_df,val_df=train_test_split(df,test_size=0.2)

In [10]:
import pandas as pd
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg19 import preprocess_input
from sklearn.model_selection import train_test_split


# Define image dimensions and batch size
img_width, img_height = 224, 224
batch_size = 32

# Data augmentation for the training set
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Data augmentation for the validation set (only rescale)
val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rescale=1.0 / 255)

# Create train and validation generators
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='path',
    y_col='label',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='path',
    y_col='label',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

Found 38177 validated image filenames belonging to 2 classes.
Found 9545 validated image filenames belonging to 2 classes.


In [12]:
# Load the pre-trained VGG19 model
base_model = VGG19(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)  # Adjust input_shape to match your data
)

# Create a new model by adding a few layers on top of the pre-trained model
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Freeze the weights of the pre-trained model
model.layers[0].trainable = False

# Compile the model with appropriate loss function, optimizer and metrics
model.compile(
    loss='binary_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)

# Print the summary of the model
model.summary()

# Define callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=5, min_lr=1e-5)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True)

# Train the model using the training data generator, validation data generator, and callbacks

history = model.fit(
        train_generator,
        epochs=50,
        validation_data=val_generator,
        callbacks=[reduce_lr, early_stopping, model_checkpoint])


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 flatten_1 (Flatten)         (None, 25088)             0         
                                                                 
 dropout_1 (Dropout)         (None, 25088)             0         
                                                                 
 dense_1 (Dense)             (None, 1)                 25089     
                                                                 
Total params: 20,049,473
Trainable params: 25,089
Non-trainable params: 20,024,384
_________________________________________________________________
Epoch 1/50
1194/1194 [==============================] - 726s 598ms/step - loss: 0.4524 - accuracy: 0.7879 - val_loss: 0.2517 - val_accuracy: 0.8977 - lr: 0.0010
Epoch 2/